# インポート

In [9]:
from pathlib import Path
import json

## issue_numberを取得

In [10]:
from pathlib import Path
import json

def read_issue_number():
    """
    issue_numberをjsonで取得する関数
    """
    # このファイル自身の場所（scripts/）
    base_dir = Path(__file__).resolve().parent
    # /boatrace/db/scrape_race_schedule_dict.json
    json_path = base_dir.parent / "db" / "issue_number.json"
    with open(json_path, "r", encoding="utf-8") as f:
        issue_number_list = json.load(f)
    return issue_number_list

## テーブルの取得

In [11]:
import urllib.request as req

url = "https://takarakuji.rakuten.co.jp/backnumber/loto6/202601/"
html = req.urlopen(url).read()

In [12]:
import pandas as pd

pd.read_html(html, encoding="utf-8")

[         回号        第2065回      第2065回.1      第2065回.2      第2065回.3  \
 0      抽せん日    2026/01/05    2026/01/05    2026/01/05    2026/01/05   
 1       本数字             6            18            21            31   
 2    ボーナス数字           (3)           NaN           NaN           NaN   
 3        1等          該当なし          該当なし          該当なし          該当なし   
 4        2等            5口            5口            5口   15,198,300円   
 5        3等          215口          215口          215口      381,700円   
 6        4等       10,741口       10,741口       10,741口        8,000円   
 7        5等      178,338口      178,338口      178,338口        1,000円   
 8  キャリーオーバー  259,218,384円  259,218,384円  259,218,384円  259,218,384円   
 
        第2065回.4      第2065回.5  
 0    2026/01/05    2026/01/05  
 1            37            40  
 2           NaN           NaN  
 3          該当なし          該当なし  
 4   15,198,300円   15,198,300円  
 5      381,700円      381,700円  
 6        8,000円        8,000円  
 7        1,00

In [13]:
import json

with open("../db/issue_number.json", "rb") as f:
    issue_number_list = json.load(f)

In [14]:
issue_number_list

['202601',
 '202512',
 '202511',
 '202510',
 '202509',
 '202508',
 '202507',
 '202506',
 '202505',
 '202504',
 '202503',
 '202502',
 '1961-1969',
 '1941-1960',
 '1921-1940',
 '1901-1920']

In [15]:
from pathlib import Path

HTML_DIR = Path("..", "data", "html")
HTML_DIR

PosixPath('../data/html')

In [16]:
import time

for issue_number in issue_number_list:
    if "-" not in issue_number:
        url = f"https://takarakuji.rakuten.co.jp/backnumber/loto6/{issue_number}/"
    else:
        url = f"https://takarakuji.rakuten.co.jp/backnumber/loto6_detail/{issue_number}/"
    html = req.urlopen(url).read()
    time.sleep(1)
    with open(HTML_DIR / f"{issue_number}.bin", "wb") as f:
        f.write(html)

In [17]:
from tqdm.notebook import tqdm

def write_html(issue_number_list, save_dir: Path):
    html_path_list = []
    for issue_number in tqdm(issue_number_list):
        filepath = save_dir / f"{issue_number}.bin"
        # binファイルがすでに存在する場合はスキップする
        if filepath.is_file():
            print(f"skipped: {issue_number}")
        else:
            if "-" not in issue_number:
                url = f"https://takarakuji.rakuten.co.jp/backnumber/loto6/{issue_number}/"
            else:
                url = f"https://takarakuji.rakuten.co.jp/backnumber/loto6_detail/{issue_number}/"
            html = req.urlopen(url).read()
            time.sleep(1)
            with open(filepath, "wb") as f:
                f.write(html)
            html_path_list.append(filepath)
    return html_path_list

In [18]:
html_path_list = write_html(issue_number_list, save_dir=HTML_DIR)

  0%|          | 0/16 [00:00<?, ?it/s]

skipped: 202601
skipped: 202512
skipped: 202511
skipped: 202510
skipped: 202509
skipped: 202508
skipped: 202507
skipped: 202506
skipped: 202505
skipped: 202504
skipped: 202503
skipped: 202502
skipped: 1961-1969
skipped: 1941-1960
skipped: 1921-1940
skipped: 1901-1920


In [19]:
html_path_list

[]

In [232]:
html_path_list = list(HTML_DIR.glob("*"))
len(html_path_list)

95

In [235]:
df3 = {}
with open('../data/html/0521-0540.bin', "rb") as f:
    html = f.read()
    dfs = pd.read_html(html)

    # 「抽せん日 / 抽選日」を含む table だけ抽出
    indexes = []
    for i, df in enumerate(dfs):
        if df.astype(str).apply(lambda x: x.str.contains("抽せん日|抽選日", na=False)).any().any():
            indexes.append(i)

    # 条件に合うものがあればそれだけ、なければ全部
    if indexes:
        target_dfs = [dfs[i] for i in indexes]
    else:
        target_dfs = dfs

    # index[回号]を設定、テーブルのを結合
    for df in target_dfs:
        # 「1等」がデータ内に含まれているか判定
        if df.astype(str).apply(lambda x: x.str.contains("1等", na=False)).any().any():
            df2 = df.drop(index=range(3, 9), errors="ignore")
            kaigou_id = df2.columns[1]
            lottery_day = df2.iat[0, 2]
            insert = df2.columns[0]
            df2.loc[df2[insert].astype(str).str.strip() == "本数字", insert] = lottery_day
            bonus_number = df2.iat[2, 1].strip("()")
            df2['ボーナス数字'] = bonus_number
            df2 = df2.drop(index=range(0, 1), errors="ignore")
            df2 = df2.drop(index=range(2, 3), errors="ignore")
            df2.columns = ['抽せん日', 'nm1', 'nm2', 'nm3', 'nm4', 'nm5', 'nm6', 'ボーナス数字']
            df2.insert(0, "回号", kaigou_id)
            df2 = df2.reset_index(drop=True)
            df3[kaigou_id] = df2
        else:
            kaigou_id = '回号'
            df.columns = ['回号', '抽せん日', 'nm1', 'nm2', 'nm3', 'nm4', 'nm5', 'nm6', 'ボーナス数字']
            df3[kaigou_id] = df
concat_df = pd.concat(df3.values())
concat_df

,回号,抽せん日,nm1,nm2,nm3,nm4,nm5,nm6,ボーナス数字
0,第0521回,2010/11/04,1,8,15,21,23,30,33
1,第0522回,2010/11/11,3,6,16,23,25,43,37
2,第0523回,2010/11/18,7,8,26,27,41,43,16
3,第0524回,2010/11/25,7,11,14,18,35,40,43
4,第0525回,2010/12/02,1,11,17,28,33,38,20
5,第0526回,2010/12/09,8,14,18,19,26,27,23
6,第0527回,2010/12/16,5,7,12,23,39,41,9
7,第0528回,2010/12/23,2,5,12,21,28,32,41
8,第0529回,2010/12/30,6,19,21,24,31,41,7
9,第0530回,2011/01/06,1,10,19,30,33,36,35


In [237]:
df3 = {}
for html_path in html_path_list:
    with open(html_path, "rb") as f:
        html = f.read()
        dfs = pd.read_html(html)

        # 「抽せん日 / 抽選日」を含む table だけ抽出
        indexes = []
        for i, df in enumerate(dfs):
            if df.astype(str).apply(lambda x: x.str.contains("抽せん日|抽選日", na=False)).any().any():
                indexes.append(i)

        # 条件に合うものがあればそれだけ、なければ全部
        if indexes:
            target_dfs = [dfs[i] for i in indexes]
        else:
            target_dfs = dfs

        # index[回号]を設定、テーブルのを結合
        for df in target_dfs:
            # 「1等」がデータ内に含まれているか判定
            if df.astype(str).apply(lambda x: x.str.contains("1等", na=False)).any().any():
                df2 = df.drop(index=range(3, 9), errors="ignore")
                kaigou_id = df2.columns[1]
                lottery_day = df2.iat[0, 2]
                insert = df2.columns[0]
                df2.loc[df2[insert].astype(str).str.strip() == "本数字", insert] = lottery_day
                bonus_number = df2.iat[2, 1].strip("()")
                df2['ボーナス数字'] = bonus_number
                df2 = df2.drop(index=range(0, 1), errors="ignore")
                df2 = df2.drop(index=range(2, 3), errors="ignore")
                df2.columns = ['抽せん日', 'nm1', 'nm2', 'nm3', 'nm4', 'nm5', 'nm6', 'ボーナス数字']
                df2.insert(0, "回号", kaigou_id)
                df2 = df2.reset_index(drop=True)
                df3[kaigou_id] = df2
            else:
                kaigou_id = '回号'
                df.columns = ['回号', '抽せん日', 'nm1', 'nm2', 'nm3', 'nm4', 'nm5', 'nm6', 'ボーナス数字']
                df3[kaigou_id] = df
concat_df = pd.concat(df3.values())
concat_df = concat_df.reset_index(drop=True)
# 抽選日を datetime に変換
concat_df["抽せん日"] = pd.to_datetime(concat_df["抽せん日"])
# 最新順に並び替え
final_df = concat_df.sort_values("抽せん日", ascending=False).reset_index(drop=True)

In [238]:
concat_df

,回号,抽せん日,nm1,nm2,nm3,nm4,nm5,nm6,ボーナス数字
0,第0821回,2013-12-12,7,10,12,34,40,43,16
1,第0822回,2013-12-16,6,11,15,24,27,33,29
2,第0823回,2013-12-19,5,6,23,30,37,38,28
3,第0824回,2013-12-23,16,19,26,27,29,32,4
4,第0825回,2013-12-26,6,10,12,16,39,40,20
...,...,...,...,...,...,...,...,...,...
113,第2034回,2025-09-15,2,13,22,25,35,37,17
114,第2035回,2025-09-18,5,10,12,17,21,35,9
115,第2036回,2025-09-22,7,10,17,26,35,42,33
116,第2037回,2025-09-25,7,12,20,23,27,29,10


In [309]:
def create_results(html_path_list):
    df3 = {}
    for html_path in html_path_list:
        with open(html_path, "rb") as f:
            html = f.read()
            dfs = pd.read_html(html)

            # 「抽せん日 / 抽選日」を含む table だけ抽出
            indexes = []
            for i, df in enumerate(dfs):
                if df.astype(str).apply(lambda x: x.str.contains("抽せん日|抽選日", na=False)).any().any():
                    indexes.append(i)

            # 条件に合うものがあればそれだけ、なければ全部
            if indexes:
                target_dfs = [dfs[i] for i in indexes]
            else:
                target_dfs = dfs

            # index[回号]を設定、テーブルのを結合
            for df in target_dfs:
                # 「1等」がデータ内に含まれているか判定
                if df.astype(str).apply(lambda x: x.str.contains("1等", na=False)).any().any():
                    df2 = df.drop(index=range(3, 9), errors="ignore")
                    kaigou_id = df2.columns[1]
                    lottery_day = df2.iat[0, 2]
                    insert = df2.columns[0]
                    df2.loc[df2[insert].astype(str).str.strip() == "本数字", insert] = lottery_day
                    bonus_number = df2.iat[2, 1].strip("()")
                    df2['ボーナス数字'] = bonus_number
                    df2 = df2.drop(index=range(0, 1), errors="ignore")
                    df2 = df2.drop(index=range(2, 3), errors="ignore")
                    df2.columns = ['抽せん日', 'nm1', 'nm2', 'nm3', 'nm4', 'nm5', 'nm6', 'ボーナス数字']
                    df2.insert(0, "回号", kaigou_id)
                    df2 = df2.reset_index(drop=True)
                    df3[kaigou_id] = df2
                else:
                    kaigou_id = '回号'
                    df.columns = ['回号', '抽せん日', 'nm1', 'nm2', 'nm3', 'nm4', 'nm5', 'nm6', 'ボーナス数字']
                    df3[kaigou_id] = df
    concat_df = pd.concat(df3.values())
    concat_df = concat_df.reset_index(drop=True)
    return concat_df

In [307]:
results = create_results(html_path_list=html_path_list)

In [308]:
results

,回号,抽せん日,nm1,nm2,nm3,nm4,nm5,nm6,ボーナス数字
回号,,,,,,,,,
0,第0821回,2013/12/12,7,10,12,34,40,43,16
1,第0822回,2013/12/16,6,11,15,24,27,33,29
2,第0823回,2013/12/19,5,6,23,30,37,38,28
3,第0824回,2013/12/23,16,19,26,27,29,32,4
4,第0825回,2013/12/26,6,10,12,16,39,40,20
...,...,...,...,...,...,...,...,...,...
113,第2034回,2025/09/15,2,13,22,25,35,37,17
114,第2035回,2025/09/18,5,10,12,17,21,35,9
115,第2036回,2025/09/22,7,10,17,26,35,42,33


In [249]:
results.isnull().sum()

回号        0
抽せん日      0
nm1       0
nm2       0
nm3       0
nm4       0
nm5       0
nm6       0
ボーナス数字    0
dtype: int64

In [250]:
save_dir = Path("..", "data", "rawdf")

In [251]:
results.to_csv(save_dir / "results.csv", sep="\t")

### 使わない

In [320]:
import pandas as pd

df = concat_df.copy()

# 念のため：列名が 0..6 であることを前提に処理
label_col = 0
date_col  = 1
num_cols  = [1, 2, 3, 4, 5, 6]

def to_one_row(group: pd.DataFrame) -> pd.Series:
    # 0列のラベルを文字列化（空白があっても耐える）
    labels = group[label_col].astype(str).str.strip()

    # 抽せん日（=抽選日として出力）
    date = group.loc[labels.isin(["抽せん日", "抽選日"]), date_col].iloc[0]

    # 本数字（1〜6列を結合）
    nums = group.loc[labels == "本数字", num_cols].iloc[0].astype(str).tolist()
    nums = [x for x in nums if x != "nan"]  # 念のため
    main_numbers = ",".join(nums)

    # ボーナス数字（1列目に入ってる）
    bonus = int(str(group.loc[labels == "ボーナス数字", date_col].iloc[0]).strip("()"))


    return pd.Series({"抽選日": date, "本数字": main_numbers, "ボーナス数字": bonus})

result_df = df.groupby(df.index, sort=False).apply(to_one_row).reset_index(names="回号")
result_df


,回号,抽選日,本数字,ボーナス数字
0,第2065回,2026/01/05,"6,18,21,31,37,40",3
1,第2066回,2026/01/08,"8,28,30,32,37,38",5
2,第1970回,2025/02/03,"11,26,27,31,36,37",42
3,第1971回,2025/02/06,"2,6,9,26,32,36",23
4,第1972回,2025/02/10,"3,14,27,28,32,36",12
...,...,...,...,...
92,第2034回,2025/09/15,"2,13,22,25,35,37",17
93,第2035回,2025/09/18,"5,10,12,17,21,35",9
94,第2036回,2025/09/22,"7,10,17,26,35,42",33
95,第2037回,2025/09/25,"7,12,20,23,27,29",10


In [327]:
nums_df = result_df["本数字"].str.split(",", expand=True)

# 列名 num1〜num6
nums_df.columns = [f"mn{i}" for i in range(1, nums_df.shape[1] + 1)]

# 数値化
nums_df = nums_df.astype(int)

# ===== ③ すべて結合 =====
final_df2 = pd.concat(
    [
        result_df[["回号", "抽選日", "ボーナス数字"]],
        nums_df
    ],
    axis=1
)

final_df2 = final_df2[
    ["回号", "抽選日", "mn1", "mn2", "mn3", "mn4", "mn5", "mn6", "ボーナス数字"]
]

final_df2

,回号,抽選日,mn1,mn2,mn3,mn4,mn5,mn6,ボーナス数字
0,第2065回,2026/01/05,6,18,21,31,37,40,3
1,第2066回,2026/01/08,8,28,30,32,37,38,5
2,第1970回,2025/02/03,11,26,27,31,36,37,42
3,第1971回,2025/02/06,2,6,9,26,32,36,23
4,第1972回,2025/02/10,3,14,27,28,32,36,12
...,...,...,...,...,...,...,...,...,...
92,第2034回,2025/09/15,2,13,22,25,35,37,17
93,第2035回,2025/09/18,5,10,12,17,21,35,9
94,第2036回,2025/09/22,7,10,17,26,35,42,33
95,第2037回,2025/09/25,7,12,20,23,27,29,10


In [286]:
# 抽選日を datetime に変換
final_df2["抽選日"] = pd.to_datetime(final_df2["抽選日"])

# 古い順に並び替え
final_df2 = final_df2.sort_values("抽選日", ascending=False).reset_index(drop=True)
final_df2

,回号,抽選日,mn1,mn2,mn3,mn4,mn5,mn6,ボーナス数字
0,第2066回,2026-01-08,8,28,30,32,37,38,5
1,第2065回,2026-01-05,6,18,21,31,37,40,3
2,第2064回,2025-12-29,2,7,24,37,39,41,3
3,第2063回,2025-12-25,4,28,29,30,38,42,15
4,第2062回,2025-12-22,1,9,18,24,35,42,8
...,...,...,...,...,...,...,...,...,...
92,第1974回,2025-02-17,2,6,13,21,24,30,43
93,第1973回,2025-02-13,7,10,14,27,28,32,4
94,第1972回,2025-02-10,3,14,27,28,32,36,12
95,第1971回,2025-02-06,2,6,9,26,32,36,23


In [333]:
def create_results_process(group: pd.DataFrame):
    df = results.copy()
    # 念のため：列名が 0..6 であることを前提に処理
    label_col = 0
    date_col  = 1
    num_cols  = [1, 2, 3, 4, 5, 6]

    kaigou = group.index[0]

    # 0列のラベルを文字列化
    labels = group[label_col].astype(str).str.strip()

    # 抽せん日（=抽選日として出力）
    date = group.loc[labels.isin(["抽せん日", "抽選日"]), date_col].iloc[0]

    # 本数字（1〜6列を結合）
    nums = group.loc[labels == "本数字", num_cols].iloc[0].astype(str).tolist()
    nums = [x for x in nums if x != "nan"]  # 念のため
    main_numbers = ",".join(nums)

    # ボーナス数字（1列目に入ってる）
    bonus = int(str(group.loc[labels == "ボーナス数字", date_col].iloc[0]).strip("()"))

    return pd.Series({
            "回号": kaigou,
            "抽選日": date,
            "ボーナス数字": bonus,
            "num1": nums[0],
            "num2": nums[1],
            "num3": nums[2],
            "num4": nums[3],
            "num5": nums[4],
            "num6": nums[5],
        })


In [334]:
results = create_results_process(group=pd.DataFrame)
results

TypeError: 'pandas._libs.properties.AxisProperty' object is not subscriptable

In [ ]:
# 本数字を各行に分解
    processed_df = result_df["本数字"].str.split(",", expand=True)

    # 列名 mn1〜mn6
    processed_df.columns = [f"mn{i}" for i in range(1, processed_df.shape[1] + 1)]

    # 数値化
    processed_df = processed_df.astype(int)

    # 結合
    final_df = pd.concat(
        [
            result_df[["回号", "抽選日", "ボーナス数字"]],
            processed_df
        ],
        axis=1
    )

    # 並び替え
    final_df = final_df[
        ["回号", "抽選日", "mn1", "mn2", "mn3", "mn4", "mn5", "mn6", "ボーナス数字"]
    ]

    # 抽選日を datetime に変換
    final_df["抽選日"] = pd.to_datetime(final_df["抽選日"])

    # 最新順に並び替え
    final_df = final_df.sort_values("抽選日", ascending=False).reset_index(drop=True)
    return final_df

In [356]:
import pandas as pd
import urllib.request as req
from bs4 import BeautifulSoup

url = "https://takarakuji.rakuten.co.jp/backnumber/loto6_detail/1961-1969/"
html = req.urlopen(url).read()
dfs = pd.read_html(html, encoding="utf-8")

# "抽選日"があるテーブルのみを抽出
count = 0
indexes = []
for i, df in enumerate(dfs):
    if df.astype(str).apply(lambda x: x.str.contains("抽せん日")).any().any():
        count += 1
        indexes.append(i)
target_dfs = [dfs[i] for i in indexes]
target_dfs

IndexError: list index out of range

In [102]:
processed_dfs = []

for df in target_dfs:
    df2 = df.drop(index=range(3, 9), errors="ignore")
    processed_dfs.append(df2)
processed_dfs[0]

,回号,第2065回,第2065回.1,第2065回.2,第2065回.3,第2065回.4,第2065回.5
0,抽せん日,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05
1,本数字,6,18,21,31,37,40
2,ボーナス数字,(3),NaN,NaN,NaN,NaN,NaN


In [95]:
race_id = processed_dfs[0].columns[1]   # 第2065回

df2 = processed_dfs[0].copy()
df2.columns = df2.iloc[0]
df2 = df2.iloc[1:].reset_index(drop=True)
df2.index = [race_id] * len(df2)
df2


,抽せん日,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05
第2065回,本数字,6,18,21,31,37,40
第2065回,ボーナス数字,(3),NaN,NaN,NaN,NaN,NaN


In [354]:
url = "https://takarakuji.rakuten.co.jp/backnumber/loto6/202601/"
html = req.urlopen(url).read()
dfs = pd.read_html(html, encoding="utf-8")
dfs[0]

,回号,第2065回,第2065回.1,第2065回.2,第2065回.3,第2065回.4,第2065回.5
0,抽せん日,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05
1,本数字,6,18,21,31,37,40
2,ボーナス数字,(3),NaN,NaN,NaN,NaN,NaN
3,1等,該当なし,該当なし,該当なし,該当なし,該当なし,該当なし
4,2等,5口,5口,5口,"15,198,300円","15,198,300円","15,198,300円"
5,3等,215口,215口,215口,"381,700円","381,700円","381,700円"
6,4等,"10,741口","10,741口","10,741口","8,000円","8,000円","8,000円"
7,5等,"178,338口","178,338口","178,338口","1,000円","1,000円","1,000円"
8,キャリーオーバー,"259,218,384円","259,218,384円","259,218,384円","259,218,384円","259,218,384円","259,218,384円"
